In [88]:
import pandas as pd
import warnings
from pandas.core.common import SettingWithCopyWarning

In [58]:


def timestamp(date):
    return pd.Timestamp(date)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

df = pd.read_csv('stocks.csv')
tickers = {'MSFT': 'microsoft', 'TSLA': 'tesla', 'GOOGL':'google', 'AAPL':'apple'}
stockdata = {}
for ticker in tickers:
    stockdata[ticker] = df.loc[df['Stock'] == ticker]

for ticker in tickers:
    stock = stockdata[ticker]
    tn = pd.read_json('tweetsnnews.json').dropna().to_dict()
    tn = pd.DataFrame.from_dict(tn[tickers[ticker]], orient='index').transpose()
    stockdic = stock.to_dict()
    tndic = tn.to_dict()
    stockdic['Daily_Info'] = {}
    for i in stockdic['Date']:
        date = stockdic['Date'][i]
        stockdic['Daily_Info'][i] = tndic.get(timestamp(date))
    df = pd.DataFrame.from_dict(stockdic)
    filename = ticker+'.json'
    print(filename)
    df.to_json(filename)     

MSFT.json
TSLA.json
GOOGL.json
AAPL.json


In [75]:
#Preprocessing
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

def clean(text, remove_stopwords = True):
  CURRENCIES = {
    "$": "USD", "zł": "PLN", "£": "GBP", "¥": "JPY", "฿": "THB", "₡": "CRC", "₦": "NGN","₩": "KRW",
    "₪": "ILS", "₫": "VND", "€": "EUR", "₱": "PHP", "₲": "PYG", "₴": "UAH", "₹": "INR",}
  CURRENCY_REGEX = re.compile(
    "({})+".format("|".join(re.escape(c) for c in CURRENCIES.keys())))
  
  EMAIL_REGEX = re.compile(
    r"(?:^|(?<=[^\w@.)]))([\w+-](\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(\.([a-z]{2,})){1,3}(?:$|(?=\b))",
    flags=re.IGNORECASE | re.UNICODE,)

  # A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
  contractions =          {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have", "i've": "i have"}
  text = text.lower()
  if True:
    text = text.split()
    new_text = []
    for word in text:
      if word in contractions:
        new_text.append(contractions[word])
      else:
          new_text.append(word)
    text = " ".join(new_text)
        
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = EMAIL_REGEX.sub(' ',text)
  text = CURRENCY_REGEX.sub(' ',text)
  text = ' '.join([contractions[t] if t in contractions else t for t in text.split(" ")])    
  text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
  text = re.sub(r"'s\b","", text)
  text = re.sub(r'&amp;', '', text) 

  if remove_stopwords:
      text = text.split()
      stops = set(stopwords.words("english"))
      text = [w for w in text if not w in stops]
      text = " ".join(text)
  
  return text

def cleanText(text):
  if(text != None):
      text = clean(text)
  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhinav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
# Preprocess Daily Info

tickers = {'MSFT': 'microsoft', 'TSLA': 'tesla', 'GOOGL':'google', 'AAPL':'apple'}

for ticker in tickers:
    df = pd.read_json(ticker+'.json')
    dic = df.to_dict()
    df = pd.DataFrame.from_dict(dic) 
    dic['Processed_Daily_Info'] = dic['Daily_Info'].copy()
    for item in dic['Daily_Info']:
        info = dic['Daily_Info'].get(item)
        if(info != None):
            for i in info:
                cleaned = cleanText(info[i])
                dic['Processed_Daily_Info'][item][i] = cleaned
    df = pd.DataFrame.from_dict(dic) 
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    filename = ticker+'_processed.json'
    print(filename)
    df.to_json(filename) 


MSFT_processed.json
TSLA_processed.json
GOOGL_processed.json
AAPL_processed.json
